### imports

In [2]:
import pandas as pd
import re
import csv
from IPython.display import display, clear_output
import nltk 
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordDetokenizer
import string
from string import capwords
from sklearn.feature_extraction.text import CountVectorizer
import geograpy3
from cliff.api import Cliff
import json 

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/neel/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/neel/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to /Users/neel/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /Users/neel/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/neel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/neel/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### loops through abstracts and matches checks if words are a country or a state name according to a csv file containing country and state names

In [3]:
def match_countries(text):
    countries_in_abstract = []
    for word in (str(text)).split():
        if word in countries_list and word not in countries_in_abstract:
            countries_in_abstract.append(word)
#         if word in abbr_country_dict and abbr_country_dict[word] not in countries_in_abstract:
#             countries_in_abstract.append(abbr_country_dict[word])
        if word in province_to_country and province_to_country[word] not in countries_in_abstract:
            countries_in_abstract.append(province_to_country[word])
        if word in demonym_dict and demonym_dict[word] not in countries_in_abstract:
            countries_in_abstract.append(demonym_dict[word])
    return countries_in_abstract


### uses geograpy3 API to attempt to assign countries to a text input

In [4]:
def geograpy_countries(input):
    places = geograpy3.get_place_context(text = input)
    return places.countries

### fills all the "countries" column for abstracts that were not assigned a column with "NA" values

In [5]:
def fill_na(array):
    if(len(array) < 1):
        return "NA"
    else:
        return array

In [6]:
df = pd.read_csv('../results/clean_tokenize_new.csv')
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)

display(df.tail())

,original_text,text,tokenized_text,no_stopwords_tokenized_text,no_stopwords_text,tdm
65447,Conceptual uncertainties in solubility calcula...,conceptual uncertainties in solubility calcula...,"['conceptual', 'uncertainties', 'in', 'solubil...","['conceptual', 'uncertainties', 'solubility', ...",conceptual uncertainties solubility calculatio...,account actinide also analysis answers ...
65448,From political to environmental conflict in th...,from political to environmental conflict in th...,"['from', 'political', 'to', 'environmental', '...","['political', 'environmental', 'conflict', 'de...",political environmental conflict development t...,achieve activity along among aquifers ...
65449,Fissure behavior in the chihuahuan desert and ...,fissure behavior in the chihuahuan desert and ...,"['fissure', 'behavior', 'in', 'the', 'chihuahu...","['fissure', 'behavior', 'chihuahuan', 'desert'...",fissure behavior chihuahuan desert depth estim...,aquifers authors base become behavior ...
65450,Integrated study of the Judy Field Block a an...,integrated study of the judy field block a an...,"['integrated', 'study', 'of', 'the', 'judy', '...","['integrated', 'study', 'judy', 'field', 'bloc...",integrated study judy field block overpressure...,accumulation across ago allows almost ...
65451,Land farm design and management in Bolivia A c...,land farm design and management in bolivia a c...,"['land', 'farm', 'design', 'and', 'management'...","['land', 'farm', 'design', 'management', 'boli...",land farm design management bolivia casestudya...,abstract acceptable accomplished additio...


In [7]:
df = df.dropna(subset=['original_text'])

In [8]:
display(len(df.index))

65434

In [14]:
display(df.head())

,original_text,text,tokenized_text,no_stopwords_tokenized_text,no_stopwords_text,tdm
0,FRACTURING AND SUBSIDENCE OF THE LAND SURFACE ...,fracturing and subsidence of the land surface ...,"['fracturing', 'and', 'subsidence', 'of', 'the...","['fracturing', 'subsidence', 'land', 'surface'...",fracturing subsidence land surface caused with...,amount aquifers area artesian bench bo...
1,An analysis of instabilities caused by salinit...,an analysis of instabilities caused by salinit...,"['an', 'analysis', 'of', 'instabilities', 'cau...","['analysis', 'instabilities', 'caused', 'salin...",analysis instabilities caused salinity gradien...,always amplitude analyse analysis aquif...
2,Pollution of the Rhine and water supply DutchA...,pollution of the rhine and water supply dutcha...,"['pollution', 'of', 'the', 'rhine', 'and', 'wa...","['pollution', 'rhine', 'water', 'supply', 'dut...",pollution rhine water supply dutcha water supp...,activities almost approximately article ...
3,DEVELOPMENT AND IN PLACE LEACHING OF MOUNTAIN ...,development and in place leaching of mountain ...,"['development', 'and', 'in', 'place', 'leachin...","['development', 'place', 'leaching', 'mountain...",development place leaching mountain city chalc...,aid analyzed annualized approximately b...
4,The study of groundwater movement in boreholes...,the study of groundwater movement in boreholes...,"['the', 'study', 'of', 'groundwater', 'movemen...","['study', 'groundwater', 'movement', 'borehole...",study groundwater movement boreholes performed...,among aquifers authors avoiding based ...


### matches words in the abstract to countries by searching for country names, province names, and demonyms

In [15]:
abbr_country_dict = {}
countries_list = []
with open("../data/iso_list_of_countries.csv") as countries:
    reader = csv.reader(countries, delimiter=',',quotechar= '"',quoting= csv.QUOTE_MINIMAL)
    for row in reader:
        name = row[0]
        if name == "name":
            continue
        abbr = row[1]
        countries_list.append(name.lower())
        abbr_country_dict[abbr] = name.lower()

In [16]:
demonym_dict = {"hawaiian": "hawaii", "alaskan": "alaska", "indian": "india", "chinese": "china", "american": "america", "russian": "russia", "indonesian": "indonesia", "pakistani": "pakistan", "nigerian": "nigeria"}


In [17]:
province_to_country = {}
with open('../data/provinces.json') as json_file: 
        provinces = json.load(json_file) 
for p_dict in provinces:
    if "english" in p_dict:
        province_name = p_dict['english'].lower()
    else:
        province_name = p_dict['name'].lower()
    province_to_country[province_name] = abbr_country_dict[p_dict['country']]


In [18]:
print(province_to_country)

{'alabama': 'united states of america', 'alaska': 'united states of america', 'arizona': 'united states of america', 'arkansas': 'united states of america', 'california': 'united states of america', 'colorado': 'united states of america', 'connecticut': 'united states of america', 'district of columbia': 'united states of america', 'delaware': 'united states of america', 'florida': 'bolivia (plurinational state of)', 'georgia': 'united states of america', 'hawaii': 'united states of america', 'idaho': 'united states of america', 'illinois': 'united states of america', 'indiana': 'united states of america', 'iowa': 'united states of america', 'kansas': 'united states of america', 'kentucky': 'united states of america', 'louisiana': 'united states of america', 'maine': 'united states of america', 'maryland': 'united states of america', 'massachusetts': 'united states of america', 'michigan': 'united states of america', 'minnesota': 'united states of america', 'mississippi': 'united state

In [19]:
print(type(df.loc[2,'text']))

df['countries'] = df['text'].apply(lambda x: match_countries(x))
df['countries'] = df['countries'].apply(lambda x: fill_na(x))

<class 'str'>


In [20]:
display(df.head())


,original_text,text,tokenized_text,no_stopwords_tokenized_text,no_stopwords_text,tdm,countries
0,FRACTURING AND SUBSIDENCE OF THE LAND SURFACE ...,fracturing and subsidence of the land surface ...,"['fracturing', 'and', 'subsidence', 'of', 'the...","['fracturing', 'subsidence', 'land', 'surface'...",fracturing subsidence land surface caused with...,amount aquifers area artesian bench bo...,NA
1,An analysis of instabilities caused by salinit...,an analysis of instabilities caused by salinit...,"['an', 'analysis', 'of', 'instabilities', 'cau...","['analysis', 'instabilities', 'caused', 'salin...",analysis instabilities caused salinity gradien...,always amplitude analyse analysis aquif...,NA
2,Pollution of the Rhine and water supply DutchA...,pollution of the rhine and water supply dutcha...,"['pollution', 'of', 'the', 'rhine', 'and', 'wa...","['pollution', 'rhine', 'water', 'supply', 'dut...",pollution rhine water supply dutcha water supp...,activities almost approximately article ...,[united kingdom of great britain and northern ...
3,DEVELOPMENT AND IN PLACE LEACHING OF MOUNTAIN ...,development and in place leaching of mountain ...,"['development', 'and', 'in', 'place', 'leachin...","['development', 'place', 'leaching', 'mountain...",development place leaching mountain city chalc...,aid analyzed annualized approximately b...,[united states of america]
4,The study of groundwater movement in boreholes...,the study of groundwater movement in boreholes...,"['the', 'study', 'of', 'groundwater', 'movemen...","['study', 'groundwater', 'movement', 'borehole...",study groundwater movement boreholes performed...,among aquifers authors avoiding based ...,[spain]


### check what  % of values in the "countries" column have been filled

In [21]:
country_list = df['countries'].tolist()
count = 0 
for country in country_list:
    if len(country) > 0:
        count +=1
print(count)
print(len(df.index))
print(count/len(df.index))
        

65434
65434
1.0


In [22]:
print(geograpy_countries("I like China and India"))

['British Indian Ocean Territory', 'China', 'United States', 'Russia']


In [23]:
display(df.head())

,original_text,text,tokenized_text,no_stopwords_tokenized_text,no_stopwords_text,tdm,countries
0,FRACTURING AND SUBSIDENCE OF THE LAND SURFACE ...,fracturing and subsidence of the land surface ...,"['fracturing', 'and', 'subsidence', 'of', 'the...","['fracturing', 'subsidence', 'land', 'surface'...",fracturing subsidence land surface caused with...,amount aquifers area artesian bench bo...,NA
1,An analysis of instabilities caused by salinit...,an analysis of instabilities caused by salinit...,"['an', 'analysis', 'of', 'instabilities', 'cau...","['analysis', 'instabilities', 'caused', 'salin...",analysis instabilities caused salinity gradien...,always amplitude analyse analysis aquif...,NA
2,Pollution of the Rhine and water supply DutchA...,pollution of the rhine and water supply dutcha...,"['pollution', 'of', 'the', 'rhine', 'and', 'wa...","['pollution', 'rhine', 'water', 'supply', 'dut...",pollution rhine water supply dutcha water supp...,activities almost approximately article ...,[united kingdom of great britain and northern ...
3,DEVELOPMENT AND IN PLACE LEACHING OF MOUNTAIN ...,development and in place leaching of mountain ...,"['development', 'and', 'in', 'place', 'leachin...","['development', 'place', 'leaching', 'mountain...",development place leaching mountain city chalc...,aid analyzed annualized approximately b...,[united states of america]
4,The study of groundwater movement in boreholes...,the study of groundwater movement in boreholes...,"['the', 'study', 'of', 'groundwater', 'movemen...","['study', 'groundwater', 'movement', 'borehole...",study groundwater movement boreholes performed...,among aquifers authors avoiding based ...,[spain]


In [26]:
print(df.loc[4, 'text'])

the study of groundwater movement in boreholes performed at the site for the future beninar dam spain using radioactive tracersin the study of sites for the construction of new dams it is necessary to know among other characteristics of the soil the following onesi ground impermeability at site to be sure of the resistance and stability of the dam ii basin impermeability for the stored waters avoiding uncontrollated losses a valuable information on this subject may be obtained through the study of waters movement at the saturated zone the method here described is based on marking the water column with a radioactive isotope and has been investigated and employed by the authors in more than  boreholes the purpose of thos e experiences was in most of the cases to know the behaviour of the groundwater aquifers as well as their recharge conditions exploitability and more importants parameters in similar manner has been successfully used at boreholes performed for the geological research of 

In [27]:
df.to_csv("../results/abstracts_cleaned_tokenized_geo_new_oct_5.csv",index=False)